# Objetivo

Aplicar o K Nearest Neighbors Na base toda. 

# Método

A partir do conjunto reduzido de features (39) 100k rows, aplicar:
- fillna <b> OK </b>

- LabelEncoder (ou algo do tipo) <b> OK </b>
- Nearest Neighbors OK
    

# Leitura dos arquivos

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Leitura da lista de features a excluir

# Carregamento Clientes

## Redução no número de features
Só rodar caso seja necessário filtrar mais vezes

In [2]:
market = pd.read_csv('../../estaticos_market.csv')
port1 = pd.read_csv('../../estaticos_portfolio1.csv')

In [3]:
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)


143

In [4]:
port1.shape

(555, 182)

In [5]:
market.drop(columns=exclude_variables, inplace= True)
market.drop(columns=['Unnamed: 0'], inplace= True)
market.set_index('id',inplace=True)
market.shape

(462298, 37)

In [6]:
port1.drop(columns=exclude_variables, inplace= True)
port1.drop(columns=['Unnamed: 0'], inplace= True)
port1.set_index('id',inplace=True)
port1.shape

(555, 37)

In [ ]:
#p1.to_csv('estaticos_49_features.csv')

# Managing NaNs


Para usar nos dados do portfólio:
- passar as strings para valor numérico
- passar bool para int
- fazer fillna para todos
- - em categoricas -> other
- - em numericas -> 0

Estamos usando o LabelEncoder

In [7]:
fill_dict = market.dtypes.to_dict()
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
fill_dict

{'fl_matriz': False,
 'de_natureza_juridica': 'other',
 'sg_uf': 'other',
 'natureza_juridica_macro': 'other',
 'de_ramo': 'other',
 'setor': 'other',
 'idade_empresa_anos': 0,
 'idade_emp_cat': 'other',
 'fl_me': False,
 'fl_sa': False,
 'fl_mei': False,
 'fl_ltda': False,
 'dt_situacao': 'other',
 'fl_st_especial': False,
 'fl_email': False,
 'fl_telefone': False,
 'fl_rm': 'other',
 'nm_divisao': 'other',
 'nm_segmento': 'other',
 'fl_optante_simples': False,
 'fl_optante_simei': False,
 'sg_uf_matriz': 'other',
 'de_saude_tributaria': 'other',
 'de_saude_rescencia': 'other',
 'nu_meses_rescencia': 0,
 'de_nivel_atividade': 'other',
 'empsetorcensitariofaixarendapopulacao': 0,
 'nm_meso_regiao': 'other',
 'nm_micro_regiao': 'other',
 'fl_passivel_iss': False,
 'qt_socios': 0,
 'idade_media_socios': 0,
 'qt_socios_feminino': 0,
 'de_faixa_faturamento_estimado': 'other',
 'de_faixa_faturamento_estimado_grupo': 'other',
 'vl_faturamento_estimado_aux': 0,
 'vl_faturamento_estimado_grupo

In [8]:
market.fillna(value=fill_dict, inplace=True)
market[cat2bool] = market[cat2bool].astype(bool)

bool2numeric = market.columns[market.dtypes == bool]
market[bool2numeric] = market[bool2numeric].astype(int)

In [9]:
#market_p1.head(2)

## Encoding

In [10]:
cat = market.columns[market.dtypes == object]
not_cat = market.columns[market.dtypes != object]

In [11]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [12]:
market_le = pd.DataFrame()
market_le[cat] = market[cat].apply(lambda col: le.fit_transform(col))
market_encoded = pd.concat([market[not_cat],market_le], axis =1)
market_encoded.dtypes

fl_matriz                                  int64
idade_empresa_anos                       float64
fl_me                                      int64
fl_sa                                      int64
fl_mei                                     int64
fl_ltda                                    int64
fl_st_especial                             int64
fl_email                                   int64
fl_telefone                                int64
fl_optante_simples                         int64
fl_optante_simei                           int64
nu_meses_rescencia                       float64
empsetorcensitariofaixarendapopulacao    float64
fl_passivel_iss                            int64
qt_socios                                float64
idade_media_socios                       float64
qt_socios_feminino                       float64
vl_faturamento_estimado_aux              float64
vl_faturamento_estimado_grupo_aux        float64
de_natureza_juridica                       int64
sg_uf               

Aqui fiz fiz um novo dataset para os ids do portifolio1 e o merge com os dados já aplicados o LabelEncoder

In [13]:
port1_le = pd.DataFrame(index=port1.index)
port1_le = market_encoded.merge(port1_le, how='right',left_index=True,right_index=True)


In [14]:
port1_le.head(3)

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9,1,0.646575,0,0,1,0,0,0,0,0,...,30,9,16,2,4,1,14,61,10,10
32e5f4e10932153a7ba869cb0386e7e02d49d2461046b866411556d2af150702,0,6.586301,0,0,0,0,0,1,1,0,...,30,9,9,4,0,2,3,44,10,7
a95d6f30bba445bd3d6b0c5b36f865b38ec01d1733609095d359649cfedf15d1,1,8.010959,0,0,1,0,0,0,0,1,...,30,9,9,1,0,2,8,0,1,1


# Nearest Neighbors em uma amostra do banco

Como não temos o que prever, usamos o treinamento não supervisionado do Nearest Neighbors



Apenas amostra por enquanto

Precisa escolher o numero de componentes a indicar

In [15]:
market_sample = market_encoded.sample(n=10000)

In [16]:
from sklearn.neighbors import NearestNeighbors

nearest = NearestNeighbors()

nearest.fit(market_sample)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

Dict para a lista de vizinhos mais próximos

In [17]:
neighbors_list = {}

for row in range(port1_le.shape[0]):
    #print(row)
    neighbors_list[row] = nearest.kneighbors(port1_le.iloc[[row]].values)

In [18]:
nearest.kneighbors(port1_le.iloc[[2]].values)

(array([[49.9944105 , 70.76741546, 73.89018255, 80.60560699, 82.29403927]]),
 array([[1469, 3319, 6702, 4990, 5061]]))

A resposta é uma lista da distância e uma lista de componentes. A componente é a linha do Dataset, ou seja, clientes.

In [19]:
neighbors_list

{0: (array([[50.19880272, 54.33234938, 55.21108284, 61.50089518, 66.44600049]]),
  array([[8433, 2623,  878, 7264, 8264]])),
 1: (array([[24632.64520767, 37306.39436511, 37333.63433254, 49087.01131419,
          49095.08370788]]), array([[6548, 2012, 1453, 8608, 9808]])),
 2: (array([[49.9944105 , 70.76741546, 73.89018255, 80.60560699, 82.29403927]]),
  array([[1469, 3319, 6702, 4990, 5061]])),
 3: (array([[37.85900068, 46.79235032, 51.52004512, 68.02895622, 73.753977  ]]),
  array([[6038, 3413, 5851, 8434, 5702]])),
 4: (array([[ 583201.25863164,  998719.86982468, 1052743.27416153,
          1063580.65694869, 1236384.99461385]]),
  array([[9389, 4264,  707, 2849, 7101]])),
 5: (array([[38.54499423, 64.38515845, 68.98593703, 78.83917219, 80.74383271]]),
  array([[ 531, 4193, 6819, 8165, 8498]])),
 6: (array([[26.15355955, 29.03463924, 35.52469047, 36.52407664, 51.01769249]]),
  array([[ 192, 6542, 7215, 4046, 2731]])),
 7: (array([[48.9653987 , 66.9144881 , 68.29722245, 69.51804649, 73

## Avaliação

Agora com o NN funcionando, precisa confrontar as sugestões para saber qual é o mais sugerido.

## Verificando o que foi mais sugerido

Da lista de vizinhos, listar em ordem os mais frequentes.

In [33]:
neighbors_list

{0: (array([[50.19880272, 54.33234938, 55.21108284, 61.50089518, 66.44600049]]),
  array([[8433, 2623,  878, 7264, 8264]])),
 1: (array([[24632.64520767, 37306.39436511, 37333.63433254, 49087.01131419,
          49095.08370788]]), array([[6548, 2012, 1453, 8608, 9808]])),
 2: (array([[49.9944105 , 70.76741546, 73.89018255, 80.60560699, 82.29403927]]),
  array([[1469, 3319, 6702, 4990, 5061]])),
 3: (array([[37.85900068, 46.79235032, 51.52004512, 68.02895622, 73.753977  ]]),
  array([[6038, 3413, 5851, 8434, 5702]])),
 4: (array([[ 583201.25863164,  998719.86982468, 1052743.27416153,
          1063580.65694869, 1236384.99461385]]),
  array([[9389, 4264,  707, 2849, 7101]])),
 5: (array([[38.54499423, 64.38515845, 68.98593703, 78.83917219, 80.74383271]]),
  array([[ 531, 4193, 6819, 8165, 8498]])),
 6: (array([[26.15355955, 29.03463924, 35.52469047, 36.52407664, 51.01769249]]),
  array([[ 192, 6542, 7215, 4046, 2731]])),
 7: (array([[48.9653987 , 66.9144881 , 68.29722245, 69.51804649, 73

In [120]:
list_size = len(neighbors_list)
num_neighbors = len(neighbors_list[0][1][0])

neighbors_idx_array = neighbors_list[0][1][0]
for line in range(1,list_size):
    neighbors_idx_array = np.concatenate((neighbors_idx_array, neighbors_list[line][1][0]), axis=None) 

if len(neighbors_idx_array) != list_size*num_neighbors:
    print("ERROR: Check array size.")
# Temos agora um array unidimensional com os índices dos clientes recomendados

In [121]:
neighbors_idx_series = pd.Series(neighbors_idx_array)

In [122]:
neighbors_idx_series.value_counts()

9652    6
572     5
6821    5
1795    5
4796    4
3191    4
6357    4
3424    4
6038    4
9931    4
3413    4
5580    4
972     4
1367    3
3426    3
7948    3
3011    3
2971    3
7127    3
4915    3
8519    3
7571    3
1275    3
3319    3
6932    3
223     3
7311    3
9474    3
7340    3
5081    3
       ..
2586    1
537     1
507     1
6616    1
6618    1
8673    1
8675    1
2532    1
2538    1
8685    1
8687    1
2546    1
4597    1
2550    1
8695    1
2556    1
533     1
6654    1
511     1
2562    1
2566    1
519     1
3164    1
521     1
8279    1
2572    1
8719    1
531     1
6676    1
6145    1
Length: 2231, dtype: int64